# Wandle Datensatz in das Format für YoloX und Yolov8 um

In [1]:
import random
import os
import shutil
import json

import numpy as np
import pandas as pd

from pycocotools import coco

## YoloX

In [37]:
datasetPathSRC = '../data/dataFiltered'
datasetPathYoloX = '../data/dataYoloX'

# Erstelle Ordnerstruktur wenn nicht vorhanden
if not os.path.exists(datasetPathYoloX):
    os.makedirs(datasetPathYoloX)
    os.makedirs(datasetPathYoloX + '/trainImages')
    os.makedirs(datasetPathYoloX + '/valImages')
    os.makedirs(datasetPathYoloX + '/annotations')

In [38]:
# Erstelle COCO Dataset categories
categories = [
    {'id': 0, 'name': 'car'},
    {'id': 1, 'name': 'pedestrian'},
    {'id': 2, 'name': 'trafficLight'},
    {'id': 3, 'name': 'truck'},
    {'id': 4, 'name': 'biker'}
    ]


In [40]:
def convertDatasetToCoco(annotationFilePath, datasetPathSRC, datasetPathYoloX, imageFolder, categories):
    """
    Konvertiert den Dataensatz in das COCO-Format
    
    annotationFilePath: Pfad zur CSV-Datei mit den Annotationen
    datasetPathSRC: Pfad zum Quell-Datensatz
    datasetPathYoloX: Pfad zum YoloX-Datensatz
    imageFolder: Ordner in dem die Bilder gespeichert werden (train oder val)
    categories: Liste mit den Kategorien

    return: COCO-Datensatz
    """
    categoryIdMap = {category['name']: category['id'] for category in categories}
    categoryData = [{'id': category['id'], 'name': category['name']} for category in categories]

    # Erstelle leeren COCO-Datensatz
    cocoDataset = coco.COCO()

    # Füge Kategorien zum COCO-Datensatz hinzu
    cocoDataset.dataset['categories'] = categoryData
    print("Categories: ", cocoDataset.dataset['categories'])

    annotationFile = pd.read_csv(annotationFilePath)
    cocoDataset.dataset['images'] = []
    cocoDataset.dataset['annotations'] = []

    for index, row in annotationFile.iterrows():
        # Pfad zum Bild
        imageFilePath = os.path.join(datasetPathSRC, row['filename'])

        # Kopiere Bild in YoloX Ordner
        imagePathYoloX = os.path.join(datasetPathYoloX, imageFolder, row['filename'])
        if not os.path.exists(imagePathYoloX):
            shutil.copy(imageFilePath, imagePathYoloX)
        
        # Füge Bilder zum COCO-Datensatz hinzu
        imageData = {
            'id': index,
            'file_name': row['filename'],
            'height': row['height'],
            'width': row['width']
        }
        cocoDataset.dataset['images'].append(imageData)

        # Füge Annotationen zum COCO-Datensatz hinzu
        annotationData = {
            'id': index,
            'image_id': index,
            'category_id': categoryIdMap[row['class']],
            'bbox': [row['xmin'], row['ymin'], row['xmax'] - row['xmin'], row['ymax'] - row['ymin']],
            'area': (row['xmax'] - row['xmin']) * (row['ymax'] - row['ymin']),
            'iscrowd': 0
        }
        cocoDataset.dataset['annotations'].append(annotationData)

    cocoDataset.createIndex()

    return cocoDataset
            
    

In [41]:
annotationFilePath = os.path.join(datasetPathSRC, '_train_annotations_newClasses.csv')
trainCocoDataset = convertDatasetToCoco(annotationFilePath, datasetPathSRC, datasetPathYoloX, 'trainImages', categories)

annotationFilePath = os.path.join(datasetPathSRC, '_val_annotations_newClasses.csv')
valCocoDataset = convertDatasetToCoco(annotationFilePath, datasetPathSRC, datasetPathYoloX, 'valImages', categories)

with open(os.path.join(datasetPathYoloX, 'annotations', 'instances_train.json'), 'w') as trainFile:
    json.dump(trainCocoDataset.dataset, trainFile)

with open(os.path.join(datasetPathYoloX, 'annotations', 'instances_val.json'), 'w') as valFile:
    json.dump(valCocoDataset.dataset, valFile)

Categories:  [{'id': 0, 'name': 'car'}, {'id': 1, 'name': 'pedestrian'}, {'id': 2, 'name': 'trafficLight'}, {'id': 3, 'name': 'truck'}, {'id': 4, 'name': 'biker'}]
creating index...
index created!
Categories:  [{'id': 0, 'name': 'car'}, {'id': 1, 'name': 'pedestrian'}, {'id': 2, 'name': 'trafficLight'}, {'id': 3, 'name': 'truck'}, {'id': 4, 'name': 'biker'}]
creating index...
index created!


## Yolov8

In [ ]:
datasetPathSRC = '../data/dataFiltered'
datasetPathYolov8 = '../data/dataYoloV8'